In [ ]:
# new code

In [6]:
'''
Import required packages
'''

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

import numpy as np
import pandas as pd

from tensorflow import keras
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector, Dropout, Input, Flatten,Concatenate
from keras.models import load_model
from keras.models import Model
from keras import optimizers
from tensorflow import optimizers

import matplotlib.pyplot as plt
import glob,os
import random
import os,shutil

In [3]:
# data preparation without lookback -- One to One prediction (o2o)
def data_prepare_o2o(dataset,scaler1, scaler2, shuffle):
    
    dataX, dataY = [], []
    scale_feature = scaler1.transform(dataset[:,:-1])
    scale_target = scaler2.transform(dataset[:,-1].reshape(-1, 1))
    for i in range(len(dataset)):
        feature = scale_feature[i]
        dataX.append(feature)
        target = scale_target[i]
        dataY.append(target)
        
    dataX, dataY = np.array(dataX), np.array(dataY)
    number_list = np.arange(0,len(dataX),1)
    random.shuffle(number_list) 
    dataX_new = []
    for index in number_list:
        dataX_new.append(dataX[index])
    dataY_new = []
    for index in number_list:
        dataY_new.append(dataY[index])

    if shuffle:
        return np.array(dataX_new), np.array(dataY_new)
    else:
        return np.array(dataX), np.array(dataY)

In [7]:
'''
Data load
'''

dataset = pd.read_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\data\data2_ML_new.csv',index_col = 0)

dynamic_feature_column = dataset.columns[8:18]
static_feature_column1 = dataset.columns[2:8]
static_feature_column2 = dataset.columns[19:]
target = dataset['NEE']

all_sites = dataset['Site'].unique()

result = pd.DataFrame(columns = all_sites, index = ['rmse_train', 'rmse_test', 'corr_train','corr_test'])

In [8]:
site_dataset = dataset.loc[dataset['Site'] == all_sites[0]]

#dynamic_feature = site_dataset[dynamic_feature_column]
#static_feature1 = site_dataset[static_feature_column1]
#static_feature2 = site_dataset[static_feature_column2]

dynamic_dataset = site_dataset.drop(columns=['Unnamed: 0.1', 'Site', 'VegType', 'lat', 'lon', 'timeStamp', 'Year',
       'Month', 'classKoppen', 'SWC', 'soilType', 'high_vegType', 'low_vegType',
       'high_vegCover', 'low_vegCover', 'Elavation'])


# split into train and test sets
train_size = int(len(dynamic_dataset.values) * 0.75)
test_size = len(dynamic_dataset.values) - train_size
train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]

# first split train/test and then apply scaler
scaler_x = StandardScaler()
scaler_x.fit(train[:,:-1])

scaler_y = StandardScaler()
scaler_y.fit(train[:,-1].reshape(-1, 1))

# train and test dataset
trainX, trainY = data_prepare_o2o(train, scaler_x, scaler_y, True)
testX, testY = data_prepare_o2o(test, scaler_x, scaler_y, True)

In [10]:
trainY.shape

(42, 1)

In [11]:
### EXP1 ###

from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

for i in range(len(all_sites)):
    site_dataset = dataset.loc[dataset['Site'] == all_sites[i]]

    #dynamic_feature = site_dataset[dynamic_feature_column]
    #static_feature1 = site_dataset[static_feature_column1]
    #static_feature2 = site_dataset[static_feature_column2]

    dynamic_dataset = site_dataset.drop(columns=['Unnamed: 0.1', 'Site', 'VegType', 'lat', 'lon', 'timeStamp', 'Year',
           'Month', 'classKoppen', 'SWC', 'soilType', 'high_vegType', 'low_vegType',
           'high_vegCover', 'low_vegCover', 'Elavation'])


    # split into train and test sets
    train_size = int(len(dynamic_dataset.values) * 0.75)
    test_size = len(dynamic_dataset.values) - train_size
    train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]
    
    # first split train/test and then apply scaler
    scaler_x = StandardScaler()
    scaler_x.fit(train[:,:-1])

    scaler_y = StandardScaler()
    scaler_y.fit(train[:,-1].reshape(-1, 1))
    
    # train and test dataset
    trainX, trainY = data_prepare_o2o(train, scaler_x, scaler_y, True)
    testX, testY = data_prepare_o2o(test, scaler_x, scaler_y, True)
    
    '''
    Valina LSTM
    '''
    # define customer optimizer
    #RMSprop= optimizers.RMSprop(lr=0.0001)
    # define special callbacks
    #reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    # define early stop
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0)

    model = Sequential()
    model.add(LSTM(32, input_shape=(trainX.shape[1], 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=300, validation_split=0.1, batch_size=5, callbacks=[early_stopping], verbose=0)
    
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY = scaler_y.inverse_transform(trainY)
    testPredict = scaler_y.inverse_transform(testPredict)
    testY = scaler_y.inverse_transform(testY)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY.squeeze(), trainPredict[:,0]))
    #print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY.squeeze(), testPredict[:,0]))
    #print('Test Score: %.2f RMSE' % (testScore))
    # calculate correlation score
    corr_train, _ = pearsonr(trainY.squeeze(), trainPredict[:,0])
    corr_test, _ = pearsonr(testY.squeeze(), testPredict[:,0])
    
    result[all_sites[i]] = [trainScore, testScore, corr_train, corr_test]
    
    print (all_sites[i])
    

AR-SLu
AT-Neu
AU-ASM
AU-Ade
AU-Cpr
AU-Cum
AU-DaP
AU-DaS
AU-Dry
AU-Emr
AU-Fog
AU-GWW
AU-Gin
AU-How
AU-Lox
AU-RDF
AU-Rig
AU-Rob
AU-Stp
AU-TTE
AU-Tum
AU-Wac
AU-Whr
AU-Wom
AU-Ync
BE-Bra
BE-Lon
BE-Vie
BR-Sa3
CA-Gro
CA-Obs
CA-Qfo
CA-SF1
CA-SF2
CA-SF3
CA-TP1
CA-TP2
CA-TP3
CA-TP4
CA-TPD
CH-Cha
CH-Dav
CH-Fru
CN-Cng
CN-Du2
CN-Du3
CN-HaM
CZ-wet
DE-Geb
DE-Gri
DE-Hai
DE-Kli
DE-Lkb
DE-Lnf
DE-Obe
DE-RuR
DE-RuS
DE-Seh
DE-SfN
DE-Tha
DE-Zrk
DK-Eng
DK-Fou
DK-Sor
ES-Amo
ES-LJu
ES-LgS
FI-Hyy
FI-Jok
FI-Let
FI-Lom
FI-Sod
FR-Gri
FR-LBr
FR-Pue
GH-Ank
GL-ZaF
IT-BCi
IT-CA1
IT-CA2
IT-CA3
IT-Col
IT-Cp2
IT-Cpz
IT-Isp
IT-Lav
IT-MBo
IT-Noe
IT-PT1
IT-Ren
IT-Ro2
IT-SR2
IT-SRo
IT-Tor
MY-PSO
NL-Hor
NL-Loo
RU-Cok
RU-Fyo
RU-Ha1
RU-SkP
RU-Tks
RU-Vrk
SD-Dem
SJ-Adv
SN-Dhr
US-AR1
US-AR2
US-ARM
US-ARb
US-ARc
US-Atq
US-Blo
US-CRT
US-Cop
US-GBT
US-GLE
US-Goo
US-IB2
US-Ivo
US-KS1
US-KS2
US-LWW
US-Lin
US-Los
US-MMS
US-Me1
US-Me2
US-Me4
US-Me5
US-NR1
US-Ne1
US-Ne2
US-Ne3
US-Oho
US-Prr


In [12]:
result.to_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\result\result_exp1_new.csv')

In [ ]:
# visualization #